<a href="https://colab.research.google.com/github/Santiago-Quinteros/Applied-Mathematics-Internship/blob/main/Compare_Ellipsoid_and_Sphere_Distances_And_Axis_Statistics_anonimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import vtk
import os
import shutil

def EllipsoidCenterAxisRotation(condyle,originRoute):
    if condyle==0:
        file='point_file_1.txt'
    if condyle==1:
        file='point_file_2.txt'
    with open(os.path.join(originRoute,file), "r") as f:
        content = f.read()
    content=content.split()

    #Center
    C=[0,0,0]
    C[0]=(float(content[content.index('Cx:')+1]))
    C[1]=(float(content[content.index('Cy:')+1]))
    C[2]=(float(content[content.index('Cz:')+1]))

    #SemiAxis
    sA=[1,1,1]
    sA[0]=(float(content[content.index('sAu:')+1]))
    sA[1]=(float(content[content.index('sAv:')+1]))
    sA[2]=(float(content[content.index('sAw:')+1]))

    #Projections of the SemiAxis in XYZ
    sAu=[1,0,0]
    sAv=[0,1,0]
    sAw=[0,0,1]

    sAu[0]=(float(content[content.index('(Aux:')+1]))
    sAu[1]=(float(content[content.index('Auy:')+1]))
    sAu[2]=(float(content[content.index('Auz:')+1].replace(')','')))

    sAv[0]=(float(content[content.index('(Avx:')+1]))
    sAv[1]=(float(content[content.index('Avy:')+1]))
    sAv[2]=(float(content[content.index('Avz:')+1].replace(')','')))

    sAw[0]=(float(content[content.index('(Awx:')+1]))
    sAw[1]=(float(content[content.index('Awy:')+1]))
    sAw[2]=(float(content[content.index('Awz:')+1].replace(')','')))

    sAu /= np.linalg.norm(sAu)
    sAv /= np.linalg.norm(sAv)
    sAw /= np.linalg.norm(sAw)

    R = np.array([sAu, sAv, sAw]).T
    return C,sA,R

In [ ]:
def Generate(condyle,originRoute,destinationRoute):
    C,sA,R=EllipsoidCenterAxisRotation(condyle,originRoute)
    GenerateEllipsoidCenter(C,condyle,destinationRoute)
    GenerateTheEllipsoid(C,sA,R,condyle,destinationRoute)

In [ ]:
#generate Ellipsoid Center
def GenerateEllipsoidCenter(C,condyle,destinationRoute):
    ellipsoid_center = vtk.vtkSphereSource()
    ellipsoid_center.SetCenter(0,0,0)
    ellipsoid_center.SetRadius(1.0)
    ellipsoid_center.SetPhiResolution(100)
    ellipsoid_center.SetThetaResolution(100)
    ellipsoid_center.Update()

    #TranslateCenter
    translation = vtk.vtkTransform()
    translation.Translate(C)

    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputConnection(ellipsoid_center.GetOutputPort())
    transform_filter.SetTransform(translation)
    transform_filter.Update()

    output = transform_filter.GetOutput()

    if condyle==0:
        outputname="Ellipsoid_Center_Medial.vtk"
    if condyle==1:
        outputname="Ellipsoid_Center_Lateral.vtk"
    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(os.path.join(destinationRoute,outputname))
    writer.SetInputData(output)
    writer.Write()

In [ ]:
#Generate Ellipsoid
def GenerateTheEllipsoid(C,sA,R,condyle,destinationRoute):
    ellipsoid = vtk.vtkSphereSource()
    ellipsoid.SetCenter(0,0,0)
    ellipsoid.SetRadius(1.0)
    ellipsoid.SetPhiResolution(100)
    ellipsoid.SetThetaResolution(100)
    ellipsoid.Update()

    #SetAxis
    AssignAxis = vtk.vtkTransform()
    AssignAxis.Scale(sA)

    AssignAxis_filter = vtk.vtkTransformFilter()
    AssignAxis_filter.SetInputConnection(ellipsoid.GetOutputPort())
    AssignAxis_filter.SetTransform(AssignAxis)
    AssignAxis_filter.Update()
    #Rotate

    rotation_matrix = R
    matrix4x4 = np.zeros((4, 4))
    matrix4x4[:3, :3] = rotation_matrix
    matrix4x4[3, 3] = 1
    rotation = vtk.vtkTransform()
    rotation.SetMatrix(matrix4x4.flatten())

    Rotate_filter = vtk.vtkTransformFilter()
    Rotate_filter.SetInputConnection(AssignAxis_filter.GetOutputPort())
    Rotate_filter.SetTransform(rotation)
    Rotate_filter.Update()

    #TranslateCenter
    translation = vtk.vtkTransform()
    translation.Translate(C)

    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputConnection(Rotate_filter.GetOutputPort())
    transform_filter.SetTransform(translation)
    transform_filter.Update()

    output = transform_filter.GetOutput()

    if condyle==0:
        outputname="Ellipsoid_Medial.vtk"
    if condyle==1:
        outputname="Ellipsoid_Lateral.vtk"
    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(os.path.join(destinationRoute,outputname))
    writer.SetInputData(output)
    writer.Write()

In [ ]:
#Create the folders where I'll save the cuadratics

def CreateEllipseFolders(NewFolder):
    folder_path = os.path.join(os.getcwd(),'Ellipsoids') #Folders with matlab logs
    patients = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric()]
    patients.sort(key=lambda x: int(x))
    legs=['Left','Right']
    condyle=["Medial","Lateral"]

    CuadraticsDone=os.path.join(os.getcwd(), NewFolder)
    os.mkdir(CuadraticsDone)
    for p in range(len(patients)):
        os.mkdir(os.path.join(CuadraticsDone,patients[p]))
        for l in range(len(legs)):
            if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
                os.mkdir(os.path.join(CuadraticsDone,patients[p],legs[l]))

    for p in range(len(patients)):
        for l in range(len(legs)):
            if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
                for c in range(len(condyle)):
                        originRoute = os.path.join(folder_path,patients[p],legs[l],condyle[c]) #Dossier avec le maiallage
                        destinationRoute=os.path.join(CuadraticsDone,patients[p],legs[l])
                        Generate(c,originRoute,destinationRoute)

In [ ]:
def AddSpheresMeshAndSSM():
    folder_path = os.path.join(os.getcwd(),'Folders','Folders') #Folders with matlab logs
    patients = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric()]
    patients.sort(key=lambda x: int(x))
    legs=['Left','Right']
    files=['file1.vtk','file2.vtk','file3.vtk','file4.vtk','file5.vtk','file6.vtk','file7.vtk','file8.vtk','file9.vtk']

    for p in range(len(patients)):
        for l in range(len(legs)):
            if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
                route=os.path.join(folder_path,patients[p],legs[l],'registration','bSpline','bspline_proximalDeformedModel.vtk')
                destination=os.path.join(os.getcwd(),'CuadraticsDone',patients[p],legs[l])
                shutil.copy(route,destination)
                route=os.path.join(folder_path,patients[p],legs[l],'localization','inModelSpace')
                for f in range(len(files)):
                    shutil.copy(os.path.join(route,files[f]),destination)

In [ ]:
#Goes over the folder with the Ellipsoids Log and creates a set of folders with the generated Ellipse and its center
#In the folders we'll have the ellipsoids, their center. The spheres from data and its center. The meshes and the SSM for visualisation porposes
#The together will be used when working with hyperboloids

#All the program suposes we are in the folder where everything is

Newfolder='CuadraticsDone'
CreateEllipseFolders(Newfolder)


In [ ]:
#save as well the spheres, meshes and ssm. folders folder ... makecopy
AddSpheresMeshAndSSM()

In [ ]:
#Create a text file with the position of sphere and ellipsoid center for each condyle and its distance according to the 3 planes
#Also the angle of the axis

#Input:Center of the Spheres and center of the Ellipsoids

#I want to go through everything and make a .txt file with the centre of the sphere and the centre of the ellipsoids.

#I could add to the AddSpheresMeshAndSSM() function the 3 planes.

#I want to see how to open the centre sphere they gave me in .vtk to extract the centre.

In [ ]:
#I want to go through all Ellipsoids, use the function that takes out the centre and save it in a file EllipsoidCenter.txt in the folder CuadraticsDone .txt with the centre of the sphere and the centre of the ellipsoids.
def CenterEllipsoidTxt(Folder):
    folder_path = os.path.join(os.getcwd(),'Ellipsoids') #Folders with matlab logs
    patients = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric()]
    patients.sort(key=lambda x: int(x))
    legs=['Left','Right']
    condyle=["Medial","Lateral"]

    CuadraticsDone=os.path.join(os.getcwd(), Folder)


    for p in range(len(patients)):
        for l in range(len(legs)):
            if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
                for c in range(len(condyle)):
                        originRoute = os.path.join(folder_path,patients[p],legs[l],condyle[c]) #Dossier avec le maiallage
                        destinationRoute=os.path.join(CuadraticsDone,patients[p],legs[l])
                        Center,_,_=EllipsoidCenterAxisRotation(c,originRoute)
                        with open (os.path.join(destinationRoute,"Ellipsoid"+condyle[c]+"Center.txt"),"w") as f:
                            f.write(" ".join(map(str, Center)))


Folder='CuadraticsDone'
CenterEllipsoidTxt(Folder)

In [ ]:
#def CenterEllipsoidTxt(Folder):
#    folder_path = os.path.join(os.getcwd(),'Ellipsoids') #Folders with matlab logs
#    patients = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric()]
#    patients.sort(key=lambda x: int(x))
#    legs=['Left','Right']
#    condyle=["Medial","Lateral"]
#
#    CuadraticsDone=os.path.join(os.getcwd(), Folder)
#
#
#    for p in range(len(patients)):
#        for l in range(len(legs)):
#            if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
#                for c in range(len(condyle)):
#                        originRoute = os.path.join(folder_path,patients[p],legs[l],condyle[c]) #Dossier avec le maiallage
#                        destinationRoute=os.path.join(CuadraticsDone,patients[p],legs[l])
#                        Center,_,_=EllipsoidCenterAxisRotation(c,originRoute)
#                        with open (os.path.join(destinationRoute,"EllipsoidCenter.txt"),"w") as f:
#                            f.write(" ".join(map(str, Center)))
#
#
#Folder='CuadraticsDone'
#CenterEllipsoidTxt(Folder)

In [ ]:
#def CenterSphereTxt(Folder):
#    folder_path = os.path.join(os.getcwd(),Folder) #Folders with the vtk spheres
#    patients = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric()]
#    patients.sort(key=lambda x: int(x))
#    legs=['Left','Right']
#    condyle=["Medial","Lateral"]
#
#    CuadraticsDone=os.path.join(os.getcwd(), Folder)
#
#    for p in range(len(patients)):
#        for l in range(len(legs)):
#            if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
#                for c in range(len(condyle)):
#                        originRoute = os.path.join(folder_path,patients[p],legs[l]) #Dossier avec le maiallage
#                        destinationRoute=os.path.join(CuadraticsDone,patients[p],legs[l])
#                        Center=ExtractSphereCenter(c,originRoute)
#                        with open (os.path.join(destinationRoute,"Sphere"+condyle[c]+"Center.txt"),"w") as f:
#                            f.write(" ".join(map(str, Center)))

#
#
#Folder='CuadraticsDone'
#CenterSphereTxt(Folder)

In [ ]:
def ExtractCenter(shapes,condyle,originRoute):
    #si estoy en el condilo medial va a ser una cosa, en el medial otra. La entrada y la salida
    if shapes==0:
        if condyle==0:
            file='file0.vtk'
        if condyle==1:
            file='file1.vtk'
    if shapes==1:
        if condyle==0:
            file='Ellipsoid_Center_Medial.vtk'
        if condyle==1:
            file='Ellipsoid_Center_Lateral.vtk'

        #extraigo todos los valores en un arreglo, promedio cada 3 para tener el promedio en cada coordenada

    with open(os.path.join(originRoute,file), "r") as f:
        content = f.read()
        content=content.split()
        points_number = int(content[11]) # extract number of points
        coords_limit=13+points_number*3
        coords = content[13:coords_limit]
        x=coords[0::3]
        y=coords[1::3]
        z=coords[2::3]
        x=[float(value) for value in x]
        y=[float(value) for value in y]
        z=[float(value) for value in z]
        Center=[]
        Center.append(round(np.mean(x),2))
        Center.append(round(np.mean(y),2))
        Center.append(round(np.mean(z),2))

    return Center

In [ ]:
#I will make the normalised difference of the centres (i.e. the normalised distance) for ellipses and spheres.
#I will have 2 normalised vectors at the centre at the origin with the direction of each of the axes.
#I project those vectors in the planes of interest (only the components remain in those planes and I make the angle between those vectors restricted to that plane).
#If you make the angle between vectors that are in the same plane, surely it is the angle you want.

#2 vectors always define a plane. There is a formula cos(tita)=a.b/|a||b| that gives you that angle.
#By restricting those vectors to a plane, I see the angle according to that plane.
from vtk.util import numpy_support

def AngleDifference(MedialCenterSphere,LateralCenterSphere,MedialCenterEllipsoid,LateralCenterEllipsoid,plane_normal):

    normal=plane_normal
    origin=[0,0,0]
    SphereAxis = [0.0, 0.0, 0.0]
    EllipsoidAxis = [0.0, 0.0, 0.0]

    vtk.vtkMath.Subtract(MedialCenterSphere, LateralCenterSphere, SphereAxis)
    vtk.vtkMath.Normalize(SphereAxis)
    ProyectedSphereAxis = [0.0, 0.0, 0.0]
    vtk.vtkPlane.ProjectVector(SphereAxis,origin, normal, ProyectedSphereAxis)

    vtk.vtkMath.Subtract(MedialCenterEllipsoid, LateralCenterEllipsoid, EllipsoidAxis)
    vtk.vtkMath.Normalize(EllipsoidAxis)
    ProyectedEllipsoidAxis = [0.0, 0.0, 0.0]
    vtk.vtkPlane.ProjectVector(EllipsoidAxis,origin, normal, ProyectedEllipsoidAxis)


    angle = vtk.vtkMath.AngleBetweenVectors(ProyectedSphereAxis, ProyectedEllipsoidAxis)
    angle = vtk.vtkMath.DegreesFromRadians(angle)
    return angle

In [ ]:
#That creates a txt with the centre of the sphere, the centre of the ellipsoid and the distance between them according to each plane.
#For the next one use the vtk functions, sure this gives the same as some function that gives you the centre of the point cloud.
#I just added the lines with centerdistance (which I haven't done yet) so it shouldn't work.
def CenterDistanceTxt(Folder):
    AllAngles_resectionPlane=[]
    AllAngles_posteriorPlane=[]



    folder_path = os.path.join(os.getcwd(),Folder) #Folders with the vtk spheres
    patients = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric()]
    patients.sort(key=lambda x: int(x))
    legs=['Left','Right']
    condyle=["Medial","Lateral"]
    shapes=["Sphere","Ellipsoid"]
    CuadraticsDone=os.path.join(os.getcwd(), Folder)

    for p in range(len(patients)):
        for l in range(len(legs)):
            if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
                for c in range(len(condyle)):
                        originRoute = os.path.join(folder_path,patients[p],legs[l]) #Dossier avec le maiallage
                        destinationRoute=os.path.join(CuadraticsDone,patients[p],legs[l])
                        for s in range(len(shapes)):
                            if s==0 and c==0:
                                SphereMedialCenter=ExtractCenter(s,c,originRoute)
                            if s==1 and c==0:
                                EllipsoidMedialCenter=ExtractCenter(s,c,originRoute)
                            if s==0 and c==1:
                                SphereLateralCenter=ExtractCenter(s,c,originRoute)
                            if s==1 and c==1:
                                EllipsoidLateralCenter=ExtractCenter(s,c,originRoute)

                #Read the plans data
                with open (os.path.join(destinationRoute,"computeFemurLandmarks_posteriorPlane.vtk"),"r") as f:
                    normal_posteriorPlane= [float(n) for n in f.readlines()[12].split()[0:3]]

                with open (os.path.join(destinationRoute,"computeFemurLandmarks_resectionPlane.vtk"),"r") as f:
                    normal_resectionPlane= [float(n) for n in f.readlines()[12].split()[0:3]]

                with open (os.path.join(destinationRoute,"computeFemurLandmarks_cap.vtk"),"r") as f:
                    normal_cap= [float(n) for n in f.readlines()[12].split()[0:3]]

                Angle_Between_Axis_posteriorPlane=AngleDifference(SphereMedialCenter,SphereLateralCenter,EllipsoidMedialCenter,EllipsoidLateralCenter,normal_posteriorPlane)
                AllAngles_posteriorPlane.append(Angle_Between_Axis_posteriorPlane)

                Angle_Between_Axis_resectionPlane=AngleDifference(SphereMedialCenter,SphereLateralCenter,EllipsoidMedialCenter,EllipsoidLateralCenter,normal_resectionPlane)
                AllAngles_resectionPlane.append(Angle_Between_Axis_resectionPlane)
                #Angle_Between_Axis_cap=AngleDifference(SphereMedialCenter,SphereLateralCenter,EllipsoidMedialCenter,EllipsoidLateralCenter,normal_cap)

                with open (os.path.join(destinationRoute,"Centers.txt"),"w") as f:
                    f.write("SphereMedialCenter:     ")
                    f.write(" ".join(map(str, SphereMedialCenter))+"\n")
                    f.write("EllipsoidMedialCenter:  ")
                    f.write(" ".join(map(str, EllipsoidMedialCenter))+"\n")
                    f.write("SphereLateralCenter:    ")
                    f.write(" ".join(map(str, SphereLateralCenter))+"\n")
                    f.write("EllipsoidLateralCenter: ")
                    f.write(" ".join(map(str, EllipsoidLateralCenter))+"\n")
                    f.write("Angle between the Ellipsoid Axis and the Sphere Axis according to different plans (º): \n")
                    f.write("Posterior: "+str(Angle_Between_Axis_posteriorPlane)+"\n")
                    f.write("Resection: "+str(Angle_Between_Axis_resectionPlane)+"\n")
                    #f.write("CAP:       "+str(Angle_Between_Axis_cap)+"\n")

                #Crear linea que pasa por los ejes
                extra=np.array([5,5,5])
                SphereAxis = vtk.vtkLineSource()
                SphereAxis.SetPoint1(np.array(SphereMedialCenter))  # Seteamos el primer punto
                SphereAxis.SetPoint2(np.array(SphereLateralCenter))  # Seteamos el segundo punto
                SphereAxis.Update()

                writer = vtk.vtkPolyDataWriter()
                writer.SetFileName(os.path.join(destinationRoute,"SphereAxis.vtk"))
                writer.SetInputConnection(SphereAxis.GetOutputPort())
                writer.Write()

                EllipsoidAxis = vtk.vtkLineSource()
                EllipsoidAxis.SetPoint1(np.array(EllipsoidMedialCenter))  # Seteamos el primer punto
                EllipsoidAxis.SetPoint2(np.array(EllipsoidLateralCenter))  # Seteamos el segundo punto
                EllipsoidAxis.Update()

                writer = vtk.vtkPolyDataWriter()
                writer.SetFileName(os.path.join(destinationRoute,"EllipsoidAxis.vtk"))
                writer.SetInputConnection(EllipsoidAxis.GetOutputPort())
                writer.Write()

    NumberOfAngles_resectionPlane = len(AllAngles_resectionPlane)
    MeanAngle_resectionPlane = np.mean(AllAngles_resectionPlane)
    SD_MeanAngle_resectionPlane =np.std(AllAngles_resectionPlane)
    MaxAngle_resectionPlane = np.max(AllAngles_resectionPlane)
    MinAngle_resectionPlane = np.min(AllAngles_resectionPlane)

    NumberOfAngles_posteriorPlane = len(AllAngles_posteriorPlane)
    MeanAngle_posteriorPlane = np.mean(AllAngles_posteriorPlane)
    SD_MeanAngle_posteriorPlane =np.std(AllAngles_posteriorPlane)
    MaxAngle_posteriorPlane = np.max(AllAngles_posteriorPlane)
    MinAngle_posteriorPlane = np.min(AllAngles_posteriorPlane)

    with open (os.path.join(os.getcwd(),"Angle between axis.txt"),"w") as f:
        f.write("Average angle between the Ellipsoid Axis and the Sphere Axis according to different plans (º): \n")

        f.write("Number of Subjects: " + str(NumberOfAngles_resectionPlane)+"\n"+"\n" )

        f.write("Resection: \n")
        f.write("Average: "+str(round(MeanAngle_resectionPlane, 2))+"\n")
        f.write("Standar Deviation: "+str(round(SD_MeanAngle_resectionPlane, 2))+"\n")
        f.write("Max: "+str(round(MaxAngle_resectionPlane, 2))+"\n")
        f.write("Min: "+str(round(MinAngle_resectionPlane, 2))+"\n")


        f.write("\nPosterior: \n")
        f.write("Average: "+str(round(MeanAngle_posteriorPlane, 2))+"\n")
        f.write("Standar Deviation: "+str(round(SD_MeanAngle_posteriorPlane, 2))+"\n")
        f.write("Max: "+str(round(MaxAngle_posteriorPlane, 2))+"\n")
        f.write("Min: "+str(round(MinAngle_posteriorPlane, 2))+"\n")

    print(AllAngles_resectionPlane.index(np.max(AllAngles_resectionPlane)))
Folder='CuadraticsDone'
CenterDistanceTxt(Folder)


14


In [ ]:
def CenterDistance(CenterSphere,CenterEllipsoid):

def AxisAngle(CenterSphere,CenterEllipsoid):

In [ ]:
import os
destinationRoute=os.path.join(os.getcwd(),"CuadraticsDone","002","Left")
with open (os.path.join(destinationRoute,"computeFemurLandmarks_posteriorPlane.vtk"),"r") as f:
    normal= [float(n) for n in f.readlines()[12].split()[0:3]]
    print(normal)


[0.0715094, 0.99735, -0.0133985]


In [ ]:
print(np.array([1,1,1])+np.array([1,1,1]))

[2 2 2]


In [ ]:
# Average angle between the Ellipsoid Axis and the Sphere Axis according to different plans (º):
# Number of Subjects: --

# Resection:
# Average: --
# Standar Deviation: --
# Max: --
# Min: --

# Posterior:
# Average: --
# Standar Deviation: --
# Max: --
# Min: --
